In [1]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import numpy as np
import lightgbm as lgbm
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from hyperopt import fmin, rand, tpe, space_eval, STATUS_OK, Trials, hp
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import FunctionTransformer
from joblib import dump,load


## Preprocessing

In [2]:
df_train = pd.read_csv('./data_sets/train.csv')
df_test = pd.read_csv('./data_sets/test.csv')
df_train.drop_duplicates(inplace=True)
df_train.drop(columns=['Descript', 'Resolution', 'Address','DayOfWeek'], inplace=True)
df_test.drop(columns=['Address','DayOfWeek'], inplace=True)
id_test = df_test.pop('Id')

In [3]:
# df_train.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)
# df_test.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)

# imp = SimpleImputer(strategy='mean')

# for district in df_train['PdDistrict'].unique():
#     df_train.loc[df_train['PdDistrict'] == district, ['X', 'Y']] = imp.fit_transform(
#         df_train.loc[df_train['PdDistrict'] == district, ['X', 'Y']])
#     df_test.loc[df_test['PdDistrict'] == district, ['X', 'Y']] = imp.transform(
#         df_test.loc[df_test['PdDistrict'] == district, ['X', 'Y']])

In [4]:
# dump(imp, 'imputer.joblib')

In [5]:
imp = load('imputer.joblib')

c:\Users\musia\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.2.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
df_test.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)
df_train.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)

for district in df_train['PdDistrict'].unique():
    df_train.loc[df_train['PdDistrict'] == district, ['X', 'Y']] = imp.transform(
        df_train.loc[df_train['PdDistrict'] == district, ['X', 'Y']])
    df_test.loc[df_test['PdDistrict'] == district, ['X', 'Y']] = imp.transform(
        df_test.loc[df_test['PdDistrict'] == district, ['X', 'Y']])

In [7]:
y_train = df_train['Category']
df_train = df_train.drop('Category', axis=1)

In [8]:
base_df  = df_train

In [9]:

class DateFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X[self.column] = pd.to_datetime(X[self.column])
        X['day'] = X[self.column].dt.day - 1
        X['month'] = X[self.column].dt.month - 1
        X['year'] = X[self.column].dt.year
        X['hour'] = X[self.column].dt.hour
        X['quarter'] = X[self.column].dt.quarter - 1
        X['dayofyear'] = X[self.column].dt.dayofyear - 1
        X['dayofweek'] = X[self.column].dt.dayofweek
        X['is_weekend'] = (
            X[self.column].dt.dayofweek >= 5).astype(int)
        X['minute'] = X[self.column].dt.minute
        X=X.drop(columns=self.column)
        return X


class CosTransformation(BaseEstimator, TransformerMixin):
    def __init__(self, column, minus=False):
        self.period = None
        self.minus = minus
        self.column = column

    def fit(self, X, y=None):
        self.minus = -1 if self.minus else 1
        self.period = X[self.column].nunique()
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X[self.column + 'cos'] = self.minus * \
            np.cos(X[self.column] * 2 * np.pi / self.period)
        return X


class SinTransformation(BaseEstimator, TransformerMixin):
    def __init__(self, column, minus=False):
        self.period = None
        self.minus = minus
        self.column = column

    def fit(self, X, y=None):
        self.minus = -1 if self.minus else 1
        self.period = X[self.column].nunique()
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X[self.column + 'sin'] = self.minus * \
            np.sin(X[self.column] * 2 * np.pi / self.period)
        return X


class BucketingCoordinatesTransformation(BaseEstimator, TransformerMixin):
    def __init__(self, columns=['X', 'Y'], bins=10):
        self.columns = columns
        self.bins = bins

    def fit(self, X, y=None):
        self.x_mean = X[self.columns[0]].mean()
        self.y_mean = X[self.columns[1]].mean()
        self.x_std = X[self.columns[0]].std()
        self.y_std = X[self.columns[1]].std()

        self.x_min = ((X[self.columns[0]] - self.x_mean) / self.x_std).min()
        self.x_max = ((X[self.columns[0]] - self.x_mean) / self.x_std).max()
        self.y_min = ((X[self.columns[1]] - self.y_mean) / self.y_std).min()
        self.y_max = ((X[self.columns[1]] - self.y_mean) / self.y_std).max()

        self.bin_edges_x = np.linspace(self.x_min, self.x_max, self.bins)
        self.bin_edges_y = np.linspace(self.y_min, self.y_max, self.bins)

        return self

    def transform(self, X, y=None):
        X = X.copy()
        X['sector_x'] = pd.cut((X[self.columns[0]] - self.x_mean) / self.x_std,
                               bins=self.bin_edges_x, labels=False, include_lowest=True)
        X['sector_y'] = pd.cut((X[self.columns[1]] - self.y_mean) / self.y_std,
                               bins=self.bin_edges_y, labels=False, include_lowest=True)
        X = X.drop(columns=self.columns)
        return X


def drop_columns(X):
    X = X.drop(['day', 'month', 'year', 'hour', 'quarter',
               'dayofweek', 'minute', 'dayofyear'], axis=1)
    return X


drop_columns_transformer = FunctionTransformer(drop_columns)

In [10]:
cos_transformer = Pipeline([
    ('cos_transformation_hour', CosTransformation(column='hour')),
    ('cos_transformation_month', CosTransformation(column='month')),
    ('cos_transformation_quarter', CosTransformation(column='quarter')),
    ('cos_transformation_dayofyear', CosTransformation(column='dayofyear')),
    ('cos_transformation_day', CosTransformation(column='day')),
    ('cos_transformation_dayofweek', CosTransformation(column='dayofweek'))
])
sin_transformer = Pipeline([
    ('sin_transformation_hour', SinTransformation(column='hour')),
    ('sin_transformation_month', SinTransformation(column='month')),
    ('sin_transformation_quarter', SinTransformation(column='quarter')),
    ('sin_transformation_dayofyear', SinTransformation(column='dayofyear')),
    ('sin_transformation_day', SinTransformation(column='day')),
    ('sin_transformation_dayofweek', SinTransformation(column='dayofweek'))

])

base_transformer = Pipeline([
    ('date_features', DateFeatureExtractor(column='Dates')),
])
sincos_transformer = Pipeline([
    ('date_features', DateFeatureExtractor(column='Dates')),
    ('cos', cos_transformer),
    ('sin', sin_transformer),
    ('drop', drop_columns_transformer)

])

bucketing_transformer = Pipeline([
    ('date_features', DateFeatureExtractor(column='Dates')),
    ('cos', cos_transformer),
    ('sin', sin_transformer),
    ('drop', drop_columns_transformer),
    ('bucketing', BucketingCoordinatesTransformation(
        columns=['X', 'Y'], bins=10))
])

categorical_transformer_label = Pipeline(steps=[
    ('label', OrdinalEncoder())
])

### EVALUATION

In [11]:
params = {
    'colsample_bytree': 0.8399515080498189,
    'gamma': 1.8809687874683567,
    'learning_rate': 0.15019457619783694,
    'max_depth': 12,
    'min_child_weight': 9.0,
    'reg_lambda': 0.3917588749286913,
    'subsample': 0.8665760243726364,
    'objective': 'multi:softprob',
    'num_class': 39,
    'verbosity': 0,
    'eval_metric': 'mlogloss',
    'tree_method': 'hist',
    'n_estimators': 1000

}

In [12]:
df_train

,Dates,PdDistrict,X,Y
0,2015-05-13 23:53:00,NORTHERN,-122.425892,37.774599
1,2015-05-13 23:53:00,NORTHERN,-122.425892,37.774599
2,2015-05-13 23:33:00,NORTHERN,-122.424363,37.800414
3,2015-05-13 23:30:00,NORTHERN,-122.426995,37.800873
4,2015-05-13 23:30:00,PARK,-122.438738,37.771541
...,...,...,...,...
878044,2003-01-06 00:15:00,TARAVAL,-122.459033,37.714056
878045,2003-01-06 00:01:00,INGLESIDE,-122.447364,37.731948
878046,2003-01-06 00:01:00,SOUTHERN,-122.403390,37.780266
878047,2003-01-06 00:01:00,SOUTHERN,-122.390531,37.780607


In [13]:
# df_train_sin = sincos_transformer.fit_transform(df_train)
# df_test_sin = sincos_transformer.transform(df_test)

In [14]:
# dump(sincos_transformer, 'sincos.joblib')

In [ ]:
sincos_transformer=load('sincos.joblib')

In [16]:
df_train_sin = sincos_transformer.transform(df_train)

df_test_sin = sincos_transformer.transform(df_test)

In [17]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_train)

In [18]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat_label', categorical_transformer_label, ['PdDistrict']),
    ], remainder='passthrough')

In [19]:
pipeline_ = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(**params))
])

In [20]:
#pipeline_.fit(df_train_sin, y)

In [21]:
# dump(pipeline_, 'pipeline.joblib')

In [22]:
pipeline_ = load('pipeline.joblib')

In [23]:
pipeline_.predict_proba(df_test_sin)

array([[3.10408976e-03, 1.13435626e-01, 1.00998339e-04, ...,
        1.78725243e-01, 2.46105772e-02, 1.93715114e-02],
       [3.30597232e-03, 1.20100945e-01, 9.77472373e-05, ...,
        9.39417109e-02, 7.59444311e-02, 2.26078890e-02],
       [1.44766644e-03, 3.90485451e-02, 8.42447625e-05, ...,
        1.28676549e-01, 1.22515280e-02, 4.02965304e-03],
       ...,
       [2.02779146e-03, 7.43790343e-02, 1.88263471e-03, ...,
        1.84749607e-02, 2.55197398e-02, 7.57018477e-03],
       [2.65104324e-03, 7.05329850e-02, 2.42879894e-03, ...,
        1.77896507e-02, 1.58452559e-02, 1.13449674e-02],
       [1.60495308e-03, 4.55645211e-02, 2.39222264e-03, ...,
        2.28535812e-02, 1.09957401e-02, 4.22610762e-03]], dtype=float32)

In [24]:
predicts=pipeline_.predict(df_test_sin)

In [25]:
predicts

array([36, 21, 16, ..., 21, 21, 21], dtype=int64)

In [26]:
predicts = label_encoder.inverse_transform(predicts)

In [27]:
predicts

array(['VEHICLE THEFT', 'OTHER OFFENSES', 'LARCENY/THEFT', ...,
       'OTHER OFFENSES', 'OTHER OFFENSES', 'OTHER OFFENSES'], dtype=object)

In [29]:
data = {"Dates": "2015-05-10 23:59:00",
        "PdDistrict": "SOUTHERN", "X": -122.408983, "Y": 37.751987}

In [31]:
data=pd.DataFrame(data,index=[0])

In [32]:
data

,Dates,PdDistrict,X,Y
0,2015-05-10 23:59:00,SOUTHERN,-122.408983,37.751987


In [33]:
data_sin=sincos_transformer.transform(data)

In [34]:
data_sin

,PdDistrict,X,Y,is_weekend,hourcos,monthcos,quartercos,dayofyearcos,daycos,dayofweekcos,hoursin,monthsin,quartersin,dayofyearsin,daysin,dayofweeksin
0,SOUTHERN,-122.408983,37.751987,1,0.965926,-0.5,6.123234e-17,-0.736055,-0.250653,0.62349,-0.258819,0.866025,1.0,0.676922,0.968077,-0.781831


In [35]:
pipeline_.predict_proba(data_sin)

array([[1.51005143e-03, 1.36684775e-01, 1.24306811e-04, 8.09879159e-04,
        4.31540757e-02, 3.66057269e-03, 5.93324611e-03, 5.30445017e-02,
        9.90307704e-03, 3.33655364e-04, 2.44742085e-04, 5.51509031e-04,
        3.18873674e-03, 7.89245497e-03, 1.17734664e-04, 3.36861168e-03,
        1.42620668e-01, 5.55570936e-03, 4.49483807e-04, 1.84622835e-02,
        6.15460835e-02, 1.24214366e-01, 1.43289908e-05, 5.79696905e-04,
        5.82903763e-03, 6.18088208e-02, 1.03933632e-03, 2.68650334e-02,
        6.24898588e-03, 2.78612686e-04, 5.10559836e-03, 4.48776642e-04,
        3.25135924e-02, 7.65453478e-06, 4.53129970e-03, 9.70708802e-02,
        9.07558203e-02, 2.99197547e-02, 1.36122610e-02]], dtype=float32)

In [37]:
preds=pipeline_.predict(data_sin)

In [38]:
preds

array([16], dtype=int64)

In [39]:
label_encoder.inverse_transform(preds)

array(['LARCENY/THEFT'], dtype=object)